<h1>Textbook Message Authentication Code (MAC)</h1><br>
Note: This example shows the CBC-MAC principle.<br>
https://cryptography.io/en/latest/ <br>

In [13]:

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend


def calcCbcMac(data, key):
    IV = bytes(16) # 16 zero bytes
    aesCipher = Cipher(algorithms.AES(key), modes.CBC(IV), backend=default_backend())
    aesEncryptor = aesCipher.encryptor()
    
    # Zero padding
    padded_data = data + b"\x00"*(16-(len(data)%16))
    # Encrypt the data
    encrypted_data =  aesEncryptor.update(padded_data)

    # the last 16 bytes are the last block and represents the CBC-MAC
    cbc_mac = encrypted_data[-16:]
       
    return cbc_mac


Alice creates a message with CBC MAC

In [14]:
alice_secrect_aes_key = bytes.fromhex('00112233445566778899AABBCCDDEEFF')

alice_msg = b'Hello World! Hello World!'
alice_cbc_mac = calcCbcMac(alice_msg, alice_secrect_aes_key)
alice_msg_with_mac = alice_msg + alice_cbc_mac

print('   Send Message with MAC = 0x'+ alice_msg_with_mac[:-16].hex(' ').upper() 
      + ' [' + alice_msg_with_mac[-16:].hex(' ').upper() + ']')

   Send Message with MAC = 0x48 65 6C 6C 6F 20 57 6F 72 6C 64 21 20 48 65 6C 6C 6F 20 57 6F 72 6C 64 21 [A6 5A 8C 91 FD 6A D2 CF 3C CA AF 88 30 4E D7 87]


<b>Taks:</b> Try what happed if Marvin modivies the message.

In [15]:
#alice_msg_with_mac = alice_msg_with_mac.replace(b'e',b'E')

Bob also calculates the MAC and compares the calculated MAC with the received MAC

In [16]:
print('Received Message with MAC = 0x'+ alice_msg_with_mac[:-16].hex(' ').upper() 
      + ' [' + alice_msg_with_mac[-16:].hex(' ').upper() + ']')

bob_secrect_aes_key = bytes.fromhex('00112233445566778899AABBCCDDEEFF')
    
received_mac = alice_msg_with_mac[-16:]
print('Received MAC =   0x' + received_mac.hex(' ').upper() )

calculated_mac = calcCbcMac(alice_msg_with_mac[0:-16], bob_secrect_aes_key )
print('Calculated MAC = 0x' + calculated_mac.hex(' ').upper() )

if received_mac == calculated_mac:
    print('The message has not been changed :=)')
else:
    print('The message has been manipulated! =:(')

Received Message with MAC = 0x48 65 6C 6C 6F 20 57 6F 72 6C 64 21 20 48 65 6C 6C 6F 20 57 6F 72 6C 64 21 [A6 5A 8C 91 FD 6A D2 CF 3C CA AF 88 30 4E D7 87]
Received MAC =   0xA6 5A 8C 91 FD 6A D2 CF 3C CA AF 88 30 4E D7 87
Calculated MAC = 0xA6 5A 8C 91 FD 6A D2 CF 3C CA AF 88 30 4E D7 87
The message has not been changed :=)
